#Pred2Annotate
This code converts model predicted contours to txt file, suitable for RoboFlow annotation.

In [ ]:
!pip install ultralytics

In [ ]:
#Use GPU enabled environment only
from ultralytics import YOLO
import cv2
import numpy as np
import torch
from scipy.interpolate import splprep, splev
import os

# Load a pretrained YOLOv8 model
model = YOLO("/content/V8_best.pt")

# Load the image and ensure proper memory handling
image_path = "/content/S8_00005.png"  # Path to your uploaded image
image      = cv2.imread(image_path)

if image is None:
    raise ValueError(f"Image at {image_path} not found or could not be loaded.")

image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Get the dimensions of the image for normalization
img_height, img_width = image_rgb.shape[:2]

# Check for CUDA availability and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Define device

# Run the model on the image with a low confidence threshold
results = model(image_path, conf=0.10, device=device, iou=0.4,
                retina_masks=True, imgsz=image_rgb.shape[0])

# Process results and generate normalized coordinates for Roboflow
output_filename = os.path.splitext(os.path.basename(image_path))[0] + ".txt"
with open(output_filename, "w") as file:
    for result in results:
        masks = result.masks  # Masks object for segmentation masks outputs

        # Get the coordinates of the masks
        xy = masks.xy  # List of arrays with (x, y) coordinates of mask vertices

        for i, polygon in enumerate(xy):
            polygon = polygon.astype(np.float32)  # Ensure coordinates are float for normalization

            # Apply spline smoothing to the polygon coordinates
            tck, u = splprep(polygon.T, s=100.0)  # s is the smoothing factor
            new_points = splev(np.linspace(0, 1, 20), tck)
            new_points = np.stack(new_points, axis=-1)  # Shape (n_points, 2)

            # Normalize the coordinates
            normalized_coords = []
            for point in new_points:
                norm_x = point[0] / img_width
                norm_y = point[1] / img_height
                normalized_coords.append(f"{norm_x} {norm_y}")

            # Write the normalized coordinates to the text file
            file.write("0 " + " ".join(normalized_coords) + "\n")

# Explicitly release memory
del image
del image_rgb
del results
torch.cuda.empty_cache()  # Clears cache on the GPU, if being used

print(f"Annotations saved to {output_filename}")

#Training An YOLOv8 Segmentation Model

In [ ]:
!pip install ultralytics

In [ ]:
# Optional: You can use one of our data from github or can upload your dataset here.
!git clone https://github.com/sktdebnath/Lysosome_Analysis.git
!unzip -q '/content/Lysosome_Analysis/V8 Training Dataset.zip'

In [ ]:
from ultralytics import YOLO
import torch

# Check for CUDA availability and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Define device

# Initialize the model
model = YOLO('yolov8s-seg.pt')

# Define the training settings
train_settings = {
    'data'    : '/content/V8_Training_Dataset/data.yaml', # Your YAML Path.
    # Open the yaml file and change the location of train, val and test files properly.
    # For example:
    #     train: /content/V8 Training Dataset/train
    #     val: train: /content/V8 Training Dataset/valid
    'epochs'  : 200,
    'cos_lr'  : True,
    'patience': 50,
    'seed'    : 42,
    'batch'   :-1,
    'imgsz'   : 640,
    'device'  : device,
    'plots'   : True
}

# Train the model with the specified settings
model.train(**train_settings)

# Empty cache for an efficient memory management.
torch.cuda.empty_cache()

#YOLO Inference
This code batchprocess multiple files at a time. All contours are preserved in the output directory.

In [ ]:
import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
from sahi.predict import get_prediction
from scipy.interpolate import splprep, splev  # For B-spline interpolation
import time  # For measuring inference time

def process_image(image_path, output_dir, model, detection_model, device):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Image not found or could not be loaded: {image_path}")
        return

    # Measure inference time
    start_time = time.time()  # Start timer

    # Perform YOLO inference   ########################################Changed Here
    sliced_result = get_prediction(
        image,  # Input image
        detection_model
    )

    # End timer
    inference_time = (time.time() - start_time) * 1000  # Convert to milliseconds

    # Measure GPU memory usage
    gpu_memory_allocated = torch.cuda.memory_allocated() / 1e6  # Convert to MB
    gpu_memory_reserved = torch.cuda.memory_reserved() / 1e6  # Convert to MB
    gpu_memory_max = torch.cuda.max_memory_allocated() / 1e6  # Convert to MB

    # Create a copy of the original image for drawing
    output_image = image.copy()

    # Process each prediction
    for prediction in sliced_result.object_prediction_list:
        try:
            # Extract mask
            mask = prediction.mask
            mask_array = mask.bool_mask  # Access the binary mask array directly

            # Validate the mask
            if mask_array is None or mask_array.sum() == 0:
                print(f"Skipping invalid or empty mask in {image_path}.")
                continue

            # Find contours from the binary mask
            contours, _ = cv2.findContours(mask_array.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Process each contour
            for contour in contours:
                contour = contour[:, 0, :]  # Simplify contour array

                # Calculate the centroid of the contour
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                else:
                    cX, cY = 0, 0

                # Smoothen the contour using B-spline interpolation
                if len(contour) > 3:  # Ensure there are enough points for interpolation
                    tck, u = splprep(contour.T, s=100.0)  # s is the smoothing factor
                    new_points = splev(np.linspace(0, 1, 50), tck)  # Generate 50 new points
                    new_points = np.stack(new_points, axis=-1).astype(np.int32)

                    # Scale the contour points to 80% of the original size relative to the centroid
                    scaled_points = []
                    for point in new_points:
                        # Translate the point so that the centroid is at the origin
                        translated_x = point[0] - cX
                        translated_y = point[1] - cY
                        # Scale the translated point by 0.8 (80%)
                        scaled_x = translated_x * 1.
                        scaled_y = translated_y * 1.
                        # Translate the point back to the original coordinate system
                        scaled_points.append([scaled_x + cX, scaled_y + cY])
                    scaled_points = np.array(scaled_points, dtype=np.int32)

                    # Shift the scaled contour points 2% towards the bottom-right direction
                    shift_factor = 0.001  # .1% shift
                    shift_x = int((image.shape[1] - cX) * shift_factor)  # Shift in x-direction
                    shift_y = int((image.shape[0] - cY) * shift_factor)  # Shift in y-direction
                    shifted_points = scaled_points + [shift_x, shift_y]

                    # Draw the scaled, shifted, and smoothened contour
                    cv2.polylines(output_image, [shifted_points], isClosed=True, color=(0, 0, 255), thickness=2)  # Red contour
        except Exception as e:
            print(f"Skipping prediction in {image_path} due to error: {e}")
            continue

    # Add text to display the total number of instances
    text = f"Total Instances: {len(sliced_result.object_prediction_list)}"
    cv2.putText(output_image, text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)

    # Convert image from BGR to RGB for display
    output_image_rgb = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)

    # Save the annotated image to the output directory
    output_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(output_path, output_image)

    # Print GPU RAM usage, inference time, and contour count
    print(f"File: {os.path.basename(image_path)} | GPU RAM Allocated: {gpu_memory_allocated:.2f} MB | GPU RAM Reserved: {gpu_memory_reserved:.2f} MB | GPU RAM Max Allocated: {gpu_memory_max:.2f} MB | Inference Time: {inference_time:.2f} ms | Contours Detected: {len(sliced_result.object_prediction_list)}")

def run_inference_on_directory(input_dir, output_dir):
    # Check if GPU is available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # Load a pre-trained YOLO model (e.g., YOLOv8)
    model_path = '/content/V10_best.pt'  # Replace with your YOLO model path
    model      = YOLO(model_path)

    # Override default parameters
    model.overrides['retina_masks'] = True  # Set retina mask
    model.overrides['max_det']      = 600   # Set your desired maximum number of detections
    model.overrides['conf']         = 0.4   # Confidence threshold
    model.overrides['iou']          = 0.4   # IoU threshold for NMS
    model.overrides['imgsz']        = 2240  # Input image size
    model.overrides['agnostic_nms'] = True  # Class-agnostic NMS
    model.overrides['mask_ratio']   = 2     # Downsampling ratio for masks
    model.to(device)                        # Move model to GPU

    # Initialize SAHI's AutoDetectionModel with YOLO
    detection_model = AutoDetectionModel.from_pretrained(
        model_type           = 'ultralytics',  # Specify model type
        model                = model,  # Pass the modified YOLO model
        confidence_threshold = 0.4,  # Adjust confidence threshold as needed
        device               = device,  # Use GPU
    )

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Process each image in the input directory
    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.png')):
            image_path = os.path.join(input_dir, filename)
            process_image(image_path, output_dir, model, detection_model, device)

            # Clear GPU memory after processing each image
            torch.cuda.empty_cache()

# Define input and output directories
input_dir  = '/content'  # Replace with your input directory
output_dir = '/content/Output_YOLO'  # Replace with your output directory

# Run inference on all images in the input directory
run_inference_on_directory(input_dir, output_dir)

#YOLO+SAHI Inference

In [ ]:
import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
from scipy.interpolate import splprep, splev  # For B-spline interpolation
import time  # For measuring inference time

def process_image(image_path, output_dir, model, detection_model, device):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Image not found or could not be loaded: {image_path}")
        return

    # Measure inference time
    start_time = time.time()  # Start timer

    # Perform sliced inference using SAHI
    sliced_result = get_sliced_prediction(
        image,  # Input image
        detection_model,  # SAHI detection model
        slice_height=640,  # Height of each slice
        slice_width=640,  # Width of each slice
        overlap_height_ratio=0.2,  # Overlap ratio between slices
        overlap_width_ratio=0.2,
        postprocess_type="NMS"
    )

    # End timer
    inference_time = (time.time() - start_time) * 1000  # Convert to milliseconds

    # Measure GPU memory usage
    gpu_memory_allocated = torch.cuda.memory_allocated() / 1e6  # Convert to MB
    gpu_memory_reserved = torch.cuda.memory_reserved() / 1e6  # Convert to MB
    gpu_memory_max = torch.cuda.max_memory_allocated() / 1e6  # Convert to MB

    # Create a copy of the original image for drawing
    output_image = image.copy()

    # Process each prediction
    for prediction in sliced_result.object_prediction_list:
        try:
            # Extract mask
            mask = prediction.mask
            mask_array = mask.bool_mask  # Access the binary mask array directly

            # Validate the mask
            if mask_array is None or mask_array.sum() == 0:
                print(f"Skipping invalid or empty mask in {image_path}.")
                continue

            # Find contours from the binary mask
            contours, _ = cv2.findContours(mask_array.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Process each contour
            for contour in contours:
                contour = contour[:, 0, :]  # Simplify contour array

                # Calculate the centroid of the contour
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                else:
                    cX, cY = 0, 0

                # Smoothen the contour using B-spline interpolation
                if len(contour) > 3:  # Ensure there are enough points for interpolation
                    tck, u = splprep(contour.T, s=100.0)  # s is the smoothing factor
                    new_points = splev(np.linspace(0, 1, 50), tck)  # Generate 50 new points
                    new_points = np.stack(new_points, axis=-1).astype(np.int32)

                    # Scale the contour points to 90% of the original size relative to the centroid
                    scaled_points = []
                    for point in new_points:
                        # Translate the point so that the centroid is at the origin
                        translated_x = point[0] - cX
                        translated_y = point[1] - cY
                        # Scale the translated point by 0.9 (90%)
                        scaled_x = translated_x * 1.
                        scaled_y = translated_y * 1.
                        # Translate the point back to the original coordinate system
                        scaled_points.append([scaled_x + cX, scaled_y + cY])
                    scaled_points = np.array(scaled_points, dtype=np.int32)

                    # Shift the scaled contour points 2% towards the bottom-right direction
                    shift_factor = 0.001  # .1% shift
                    shift_x = int((image.shape[1] - cX) * shift_factor)  # Shift in x-direction
                    shift_y = int((image.shape[0] - cY) * shift_factor)  # Shift in y-direction
                    shifted_points = scaled_points + [shift_x, shift_y]

                    # Draw the scaled, shifted, and smoothened contour
                    cv2.polylines(output_image, [shifted_points], isClosed=True, color=(0, 0, 255), thickness=2)  # Red contour
        except Exception as e:
            print(f"Skipping prediction in {image_path} due to error: {e}")
            continue

    # Add text to display the total number of instances
    text = f"Total Instances: {len(sliced_result.object_prediction_list)}"
    cv2.putText(output_image, text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)

    # Convert image from BGR to RGB for display
    output_image_rgb = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)

    # Save the annotated image to the output directory
    output_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(output_path, output_image)

    # Print GPU RAM usage, inference time, and contour count
    print(f"File: {os.path.basename(image_path)} | GPU RAM Allocated: {gpu_memory_allocated:.2f} MB | GPU RAM Reserved: {gpu_memory_reserved:.2f} MB | GPU RAM Max Allocated: {gpu_memory_max:.2f} MB | Inference Time: {inference_time:.2f} ms | Contours Detected: {len(sliced_result.object_prediction_list)}")

def run_inference_on_directory(input_dir, output_dir):
    # Check if GPU is available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # Load a pre-trained YOLO model (e.g., YOLOv8)
    model_path = '/content/V10_best.pt'  # Replace with your YOLO model path
    model      = YOLO(model_path)

    # Override default parameters
    model.overrides['retina_masks'] = True  # Set retina mask
    model.overrides['conf']         = 0.4   # Confidence threshold
    model.overrides['iou']          = 0.4   # IoU threshold for NMS
    model.overrides['imgsz']        = 640   # Input image size
    model.overrides['agnostic_nms'] = True  # Class-agnostic NMS
    model.overrides['mask_ratio']   = 2     # Downsampling ratio for masks
    model.to(device)                        # Move model to GPU

    # Initialize SAHI's AutoDetectionModel with YOLO
    detection_model = AutoDetectionModel.from_pretrained(
        model_type           = 'ultralytics',  # Specify model type
        model                = model,  # Pass the modified YOLO model
        confidence_threshold = 0.4,  # Adjust confidence threshold as needed
        device               = device,  # Use GPU
    )

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Process each image in the input directory
    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.png')):
            image_path = os.path.join(input_dir, filename)
            process_image(image_path, output_dir, model, detection_model, device)

            # Clear GPU memory after processing each image
            torch.cuda.empty_cache()

# Define input and output directories
input_dir  = '/content'  # Replace with your input directory
output_dir = '/content/Output_SAHI'  # Replace with your output directory

# Run inference on all images in the input directory
run_inference_on_directory(input_dir, output_dir)

#YOLO vs YOLO+SAHI Detection Comparison:

In [ ]:
import cv2
import torch
import matplotlib.pyplot as plt
from ultralytics import YOLO
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
from sahi.predict import get_prediction

# Check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Load a pre-trained YOLO model (e.g., YOLOv8)
model_path = '/content/best.pt'  # Replace with your YOLO model path
model      = YOLO(model_path)

# Override default parameters
model.overrides['max_det']      = 500  # Set your desired maximum number of detections
model.overrides['iou']          = 0.4  # Set your desired IOU
model.overrides['retina_masks'] = True # Set retina mask
model.to(device)                  # Move model to GPU

# Load an image (replace with your image path)
image_path = '/content/5_frame_1.png'  # Upload your image to Colab or provide a path
image      = cv2.imread(image_path)

# Ensure the image is loaded
if image is None:
    raise FileNotFoundError(f"Image not found at {image_path}")

# Initialize SAHI's AutoDetectionModel with YOLO
detection_model = AutoDetectionModel.from_pretrained(
    model_type           = 'ultralytics',  # Specify model type
    model                = model,          # Pass the modified YOLO model
    confidence_threshold = 0.4,            # Adjust confidence threshold as needed
    device               = device          # Use GPU
)

# Perform standard YOLO prediction
YOLO_result = get_prediction(image_path, detection_model)

# Perform sliced inference using SAHI
sliced_result = get_sliced_prediction(
    image,                       # Input image
    detection_model,             # SAHI detection model
    slice_height         = 640,  # Height of each slice
    slice_width          = 640,  # Width of each slice
    overlap_height_ratio = 0.2,  # Overlap ratio between slices
    overlap_width_ratio  = 0.2
)

for prediction in YOLO_result.object_prediction_list:
    x1, y1, x2, y2 = prediction.bbox.minx, prediction.bbox.miny, prediction.bbox.maxx, prediction.bbox.maxy
    centroid_x     = int((x1 + x2) / 2)  # Calculate centroid x-coordinate
    centroid_y     = int((y1 + y2) / 2)  # Calculate centroid y-coordinate
    # Draw centroid on the image
    cv2.circle(image, (centroid_x, centroid_y), 7, (0, 255, 0), -1)  # Green dot for YOLO centroid

for prediction in sliced_result.object_prediction_list:
    x1, y1, x2, y2 = prediction.bbox.minx, prediction.bbox.miny, prediction.bbox.maxx, prediction.bbox.maxy
    centroid_x     = int((x1 + x2) / 2)  # Calculate centroid x-coordinate
    centroid_y     = int((y1 + y2) / 2)  # Calculate centroid y-coordinate
    # Draw centroid on the image
    cv2.circle(image, (centroid_x, centroid_y), 7, (0, 0, 255), -1)  # Red dot for SAHI centroid

# Add text to display the total number of centroids with YOLO and SAHI
YOLO_text = f"Total Centroids YOLO      : {len(YOLO_result.object_prediction_list)}"
SAHI_text = f"Total Centroids YOLO+SAHI: {len(sliced_result.object_prediction_list)}"

cv2.putText(image, YOLO_text, (10, 60),  cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
cv2.putText(image, SAHI_text, (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)

# Convert image from BGR to RGB for display
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Display the image with centroids
plt.figure(figsize=(10, 10))
plt.imshow(image_rgb)
plt.axis('off')
plt.title("Detected Objects with Centroids (YOLO and SAHI)")
plt.show()